In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from pandas import pivot_table

df = pd.DataFrame()
for file_path in os.listdir('results'):
    if file_path.endswith('.csv'):
        df = pd.concat([df, pd.read_csv(os.path.join('results', file_path))])

df.drop('Unnamed: 0', axis=1, inplace=True)
df['model'] = df['model'].replace(
    {'yandex': "YandexGPT-5-Lite-8B-instruct", "qwen2.5": "Qwen2.5-7B-Instruct", "ministral": "Ministral-8B-Instruct-2410"})
# df

In [ ]:
def style_plot(ax, title, ylabel):
    ax.set_title(title, fontsize=14, pad=15)
    ax.set_xlabel('')
    ax.set_ylabel(ylabel, fontsize=11)
    ax.tick_params(axis='x', rotation=0)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # добавляем метки, чтобы они были на правильных местах
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f', label_type='edge', padding=3, fontsize=9)

    current_ylim = ax.get_ylim()
    ax.set_ylim(top=current_ylim[1] * 1.1)
    

# подготовка данных, вычисляем медиану
median_tokens = df.groupby(['model', 'is_tuned'])['completion_tokens'].median().unstack()
median_tokens.rename(columns={False: 't = 1.0\ntop_p=1.0\nrep_pen=1.0', True: 't = 0.5\ntop_p=0.6\nrep_pen=1.1'}, inplace=True)

median_duration = df.groupby(['model', 'is_tuned'])['duration'].median().unstack()
median_duration.rename(columns={False: 't = 1.0\ntop_p=1.0\nrep_pen=1.0', True: 't = 0.5\ntop_p=0.6\nrep_pen=1.1'}, inplace=True)

median_tts = df.groupby(['model', 'is_tuned'])['tts'].median().unstack()
median_tts.rename(columns={False: 't = 1.0\ntop_p=1.0\nrep_pen=1.0', True: 't = 0.5\ntop_p=0.6\nrep_pen=1.1'}, inplace=True)

fig = plt.figure(figsize=(16, 12))
fig.suptitle('Сравнение моделей (медианные значения)', fontsize=20)

# первая строка: 1 график (количество токенов)
ax1 = plt.subplot(2, 1, 1)
median_tokens.plot(kind='bar', ax=ax1, width=0.4)
style_plot(ax1, 'Кол-во сгенерированных токенов', 'Кол-во токенов')
ax1.legend(title='Параметры', ncol=2)

# вторая строка: 2 графика (duration и tps)
ax2 = plt.subplot(2, 2, 3)
median_duration.plot(kind='bar', ax=ax2, width=0.4, legend=False)
style_plot(ax2, 'Время генерации', 'Время (сек)')

ax3 = plt.subplot(2, 2, 4)
median_tts.plot(kind='bar', ax=ax3, width=0.4, legend=False)
style_plot(ax3, 'TPS', 'Значение TPS')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
pivot_df = df.pivot_table(
    index=['model', 'is_tuned'],
    columns='prompt_id',
    values='output_text',
    aggfunc='first'
)

markdown_string = "| Модель | P1 (Креативное письмо) | P2 (Классификация) | P3 (Извлечение) |\n"
markdown_string += "|:---|:---|:---|:---|\n"

for (model, is_tuned), row in pivot_df.iterrows():

    status = "tune" if is_tuned else "no tune"
    row_header = f"**{model}** - {status}"

    p1_text = str(row.get('P1', '')).replace('\r\n', '<br>').replace('|', '&#124;')
    p2_text = str(row.get('P2', '')).replace('\r\n', '<br>').replace('|', '&#124;')
    p3_text = str(row.get('P3', '')).replace('\r\n', '<br>').replace('|', '&#124;')

    markdown_string += f"| {row_header} | {p1_text} | {p2_text} | {p3_text} |\n"

display(Markdown(markdown_string))
# print(markdown_string)